This code program is intended to provide a customer rewards system for frequent flyers. 

Specific use case: in store credit card purchases at a restaurant.
General case: All credit card purchases for all types of non-customizable purchase items. 

Problem: Restaurants know retention is the key to sustaining sales. Hospitality is a great driver or retention. When a customer walks in, they want to feel seen and known.  A great surprise and delight elemnet to hospitality are free gifts. Even better when they are gifts you like. 

This program intends to recognize customers by their Credit Card number. 
Evaluate it they are first timecustomers or frequent flyers. 
Provide an introductory give OR a frequent flyer gift based on their previous selections. 

Import functions:

In [767]:
from datetime import datetime
from datetime import timedelta
# datetime.strptime('11 am','%I %p')

Setup basic structure for menu, order, db:

In [768]:
# Menu class setup. Sample menus for Lunch and Dinner, each with menu item categories, menu items, prices

class Menu:
    def __init__(self,name,start_time,end_time,**menu_items):
        self.name = name
        self.start_time = start_time
        self.end_time = end_time
        self.menu_items = menu_items
        print(self.name + " initiated")
    
    def __repr__(self):
        return "This is for {} Menu Items and takes name, start time, end time, and all menu items broken down into drinks, apps, mains, and desserts with prices".format(self.name)
    
Lunch = Menu('Lunch',"11 am","2 pm",drinks={'coconut water':4,'pandan water':1}, apps={'banh bot loc':8, 'goi ga':8}, mains={'beef pho':11, 'banh mi dac biet':11}, dessert={'che':5, 'cookie':4})
Dinner = Menu('Dinner',"5 am","9 pm",drinks={'coconut water':5,'mezcal calamansi':12}, apps={'banh bot loc':11, 'goi ga':11}, mains={'beef pho':12, 'shaking beef':18}, dessert={'che':7, 'fried sesame balls':10})

# print(Lunch)
# print(Lunch.start_time)
# print(Lunch.end_time)
# print(Lunch.menu_items)
# print(Dinner.start_time)
# print(Dinner.end_time)
# print(Dinner.menu_items)
print(Lunch.menu_items.keys())

Lunch initiated
Dinner initiated
dict_keys(['drinks', 'apps', 'mains', 'dessert'])


In [1088]:
# Database <--> Order Setup:

#DB skeleton
order_database = {}

#Class and all work for Orders

class Order:
    counter = 0
    
#     Instance variables
    def __init__(self,customer_name,credit_card,menu,table_number='',**order):
        self.customer_name = customer_name
        self.credit_card = credit_card
        self.menu = menu
        self.table_number = table_number
        self.order = order
        self.time = datetime.now()
        Order.counter += 1
        self.orderid = Order.counter
        self.subtotal = 0
        self.counter = {}
        
# Calculate subtotals based on order items and Menu
        for types in [Lunch.menu_items,Dinner.menu_items]:
            if self.menu == types:
                for categories in ['mains','apps','drinks','dessert']:
                    if self.order.get(categories) != None:
                        for items in self.order.get(categories).split(", "):
                            if items in types[categories]:
                                self.subtotal += types[categories].get(items)      
#              
# Establish database record:
        order_database[self.orderid] = {'Customer Name':self.customer_name,'Credit Card':self.credit_card,'Time':self.time,'Subtotal':self.subtotal,'Order':self.order}
#         print("Order taken!")

# Print string:
    def __repr__(self):
        return "Name: {}. Time: {}. Order: {}".format(self.customer_name, self.time, self.order)


# ALL FUNCTIONS USED:

# Returns a list of all CC to match to see if they are first time customers.
    def create_list(self):
        credit_card_list = []
        for record in order_database:
            credit_card_list.append(order_database[record]['Credit Card'])
        return credit_card_list
    
# Add up the subtotals for a specific card in a specified amount of time. Default is 100.
    def db_subtotal_agg(self,time=100):
        credit_card_total = 0
        for id_num, details in order_database.items():
            if details['Credit Card'] == self.credit_card and self.time > datetime.now() - timedelta(days=100):
                credit_card_total += details['Subtotal'] 
        return credit_card_total
    
# Query all orders for the past 200 days. Find the most ordered Apps. 
    def most_liked_in_cat(self,cat='apps'):
        temp_list = []
        for key, value in order_database.items():
            if value.get('Credit Card') == self.credit_card and self.time > datetime.now() - timedelta(days=200):
                tip = value.get('Order').get(cat).split(', ')
                temp_list.extend(tip)
        comb_temp_list = temp_list

        counter_dict = {}
        for item in comb_temp_list:
            if counter_dict.get(item) == None:
                counter_dict[item]=1
            else:
                counter_dict[item] += 1
        self.counter = counter_dict
        return self.counter


# FREQUENT FLYER FUNCTIONS - Repeat customers, returns bonus gift or none. 

    def frequent_flyer(self):
# If app was ordered more than twice AND it is not on the current order, create a dictionary of compared items 
        comp_items = {}
        for key,value in self.counter.items():
            if value >2 and key != Order_3.order.get('apps'):
                comp_items[key] = value
        print(comp_items)
# Remove the free items from the DB
        comp_list_no_free = {}
        for k,v in comp_items.items():
            if 'free' in k:
                continue
            else:
                comp_list_no_free[k] = v
        print(comp_list_no_free)
# If this returns empyy, given a free sorbet
        if comp_list_no_free == {}:
            bonus_item = {'dessert':'free sorbet'}

# Get the most ordered item and return a dict. 
        else:
            for k,v in comp_list_no_free.items():
                if v == max(comp_list_no_free.values()):
                    bonus_item = {'apps':k}
                    bonus_item['apps'] = 'free '+str(k)
                    print(bonus_item)
        return bonus_item

# Add the reward to the database order. 
    def add_reward(self,added_reward):
        for key,value in added_reward.items():
            order_database[self.orderid]['Order'][key] = value + ", " + self.order[key]
        print(order_database[self.orderid]['Order'])
    

        

# START PROGRAM

    def program(self):
        print(self.customer_name)
        print(self.menu['apps'])
        bonus_item=''
# Evaluates CC list. If first time, get a free introductory item. 
# def create_list(self):
        credit_card_list = []
        for record in order_database:
            credit_card_list.append(order_database[record]['Credit Card'])
        if credit_card_list.count(self.credit_card) == 1:
            bonus_item = {'apps':'free introductory app'}
            print('First time customer')
            print(bonus_item)

# If not the first time, calculate the sum of subtotals for that credit card in the past 100 days.
        else:

# def db_subtotal_agg(self,time=100):
            credit_card_total = 0
            for id_num, details in order_database.items():
                if details['Credit Card'] == self.credit_card and self.time > datetime.now() - timedelta(days=100):
                    credit_card_total += details['Subtotal'] 
                    
# If subtotal is greater than 350:
            if credit_card_total > 50:
                temp_list = []
                for key, value in order_database.items():
                    if value.get('Credit Card') == self.credit_card and self.time > datetime.now() - timedelta(days=200):
                        tip = value.get('Order').get('apps').split(', ')
                        temp_list.extend(tip)
                print('This a combined list of apps:') 
                print(temp_list)
                counter_dict = {}
                for item in temp_list:
                    if counter_dict.get(item) == None:
                        counter_dict[item]=1
                    else:
                        counter_dict[item] += 1
                self.counter = counter_dict
                print("Rolled Up:")
                print(self.counter)
                
# If app was ordered more than twice AND it is not on the current order, create a dictionary of compared items 
                comp_items = {}
                for key,value in self.counter.items():
                    if value >2:
                        if key in self.order.get('apps'):
                            continue
                        elif 'free' in key:
                            continue
                        elif key not in self.menu['apps'].keys():
                            continue
                        else: 
                            comp_items[key] = value   
                print('Remove current order items, free items,and those less than 3')
                print(comp_items)
                
# If there no frequent items NOT in the current order, send a dessert
                if comp_items == {}:
                    bonus_item = {'dessert':'free sorbet'}
                    print('Sending dessert bc no match')
                    print(bonus_item)
                
# If there is a list of frequent items. Get the most ordered item and return a dict. 
                else:
                    for k,v in comp_items.items():
                        if v == max(comp_items.values()):
                            bonus_item = {'apps':k}
                            bonus_item['apps'] = 'free '+str(k)
                    print('Matched an item')
                    print(bonus_item)
                    
# Add to the database
# def add_reward(self,added_reward):
                    for key,value in bonus_item.items():
                        order_database[self.orderid]['Order'][key] = value + ", " + self.order[key]
                        print(order_database[self.orderid]['Order']['apps'])
# return bonus_item
            else:
                print('no rewards')
        return 'Bonus item is', bonus_item

        
Order_1 = Order('Jerry', 123456789, Lunch.menu_items, drinks='pandan water', mains = 'beef pho, banh mi dac biet', apps='banh bot loc, banh bot loc, banh bot loc, free banh bot loc, free banh bot loc, free banh bot loc', dessert = 'che, che, che')
Order_2 = Order('Mita', 123454555, Dinner.menu_items, drinks='coconut water, coconut water, mezcal calamansi', mains = 'beef pho, shaking beef', apps='goi ga, goi ga',dessert = 'che, che, che')
Order_3 = Order('Jerry', 123456789, Lunch.menu_items, drinks='pandan water', mains = 'beef pho, banh mi dac biet', apps='goi ga, goi ga, goi ga', dessert = 'che, che, che')
Order_4 = Order('Mita', 123454555, Dinner.menu_items, drinks='coconut water, coconut water, mezcal calamansi', mains = 'beef pho, shaking beef', apps='goi ga, goi ga',dessert = 'che, che, che')
Order_5 = Order('June', 555555555, Lunch.menu_items, drinks='pandan water', mains = 'beef pho, banh mi dac biet', apps='banh bot loc, banh bot loc, banh bot loc', dessert = 'che, che, che')
Order_6 = Order('Jill', 555255555, Lunch.menu_items, drinks='pandan water', mains = 'beef pho', apps = '', dessert = 'che, che, che')
Order_7 = Order('Josh', 555255555, Dinner.menu_items, drinks='coconut water', mains = 'beef pho', apps = '', dessert = '')

# # print(Order_2.orderid)
# # print(Order_2.menu)
# print(Order_1.menu)
# print(Order_1.credit_card)
# print(Order_2.subtotal)
# print(Order_5.subtotal)
# print(Order_7.subtotal)
# # print(Order_1.table_number)
# # print(Order_1.order['drinks'])
# # print(Order_1.order)
# # print(order_database)
# # print(Order_2.db_subtotal_agg())
# # # Order.credit_card_list
# # print(Order_1.add_reward({'apps':'banh bot loc', 'mains':'beef pho'}))
# print(Order_1.create_list())
# # print(order_database)
# # print(Order_1.most_liked_in_cat())
# # print(Order_1.counter)
# # print(Order_1.frequent_flyer())

print("PROGRAM1")

print(order_database[Order_1.orderid]['Order']['apps'])
print(Order_1.program())
# print(order_database[Order_1.orderid]['Order']['apps'])
# print(Order_2.program())
# print(Order_7.program())

PROGRAM1
banh bot loc, banh bot loc, banh bot loc, free banh bot loc, free banh bot loc, free banh bot loc
Jerry
{'banh bot loc': 8, 'goi ga': 8}
This a combined list of apps:
['banh bot loc', 'banh bot loc', 'banh bot loc', 'free banh bot loc', 'free banh bot loc', 'free banh bot loc', 'goi ga', 'goi ga', 'goi ga']
Rolled Up:
{'banh bot loc': 3, 'free banh bot loc': 3, 'goi ga': 3}
Remove current order items, free items,and those less than 3
{'goi ga': 3}
Matched an item
{'apps': 'free goi ga'}
free goi ga, banh bot loc, banh bot loc, banh bot loc, free banh bot loc, free banh bot loc, free banh bot loc
('Bonus item is', {'apps': 'free goi ga'})


In [945]:
#search by cc. If no order records, give them a free che dessert. 
#search by cc number and calculate sum of total for the past 3 months. If sum greater than reward threshold, give them a free item. 


In [891]:
# Match against current order. 

toot = {'banh xeo':3,'banh bot loc': 9, 'goi ga': 2, 'free dumplings': 10, 'free beef pho': 6}
Order_3 = Order('Jerry', 123456789, 'Lunch', drinks='pandan water', mains = 'beef pho, banh mi dac biet', apps='goi ga, goi ga', dessert = 'che, che, che')
print(Order_3.order.get('apps'))


# Compare toot - order history - 
# If there it was ordered more than twice AND it is not on the current order, create a dictionary of compared items 

comp_items = {}
for key,value in toot.items():
    if value >2 and key != Order_3.order.get('apps'):
        comp_items[key] = value
print(comp_items)

# Remove the free items:

comp_list_no_free = {}
for k,v in comp_items.items():
    if 'free' in k:
        continue
    else:
        comp_list_no_free[k] = v
print(comp_list_no_free)

# If this returns empyy, given a free sorbet
if comp_list_no_free == {}:
    bonus_item = {'dessert':'free sorbet'}

# Get the most ordered item and return a dict. 
else:
    for k,v in comp_list_no_free.items():
        if v == max(comp_list_no_free.values()):
            bonus_item = {'apps':k}
            bonus_item['apps'] = 'free '+str(k)
            print(bonus_item)

Order taken!
goi ga, goi ga
{'banh xeo': 3, 'banh bot loc': 9, 'free dumplings': 10, 'free beef pho': 6}
{'banh xeo': 3, 'banh bot loc': 9}
{'apps': 'free banh bot loc'}


In [823]:
print(datetime.now() - timedelta(days=100))
random_day = datetime(2023,12,29)
print(random_day > datetime.now() - timedelta(days=100))

2023-11-25 11:47:55.984787
True


In [ ]:
# Old Subtotal Code

# V1
                 
#         if self.menu == 'Lunch':
#             if self.order.get('mains') != None:
#                 for items in self.order.get('mains').split(", "):
#                     if items in Lunch.menu_items['mains']:
#                         self.subtotal += Lunch.menu_items['mains'].get(items)
#             if self.order.get('drinks') != None:
#                 for items in self.order.get('drinks').split(", "):
#                     if items in Lunch.menu_items['drinks']:
#                         self.subtotal += Lunch.menu_items['drinks'].get(items)
#             if self.order.get('apps') != None:
#                 for items in self.order.get('apps').split(", "):
#                     if items in Lunch.menu_items['apps']:
#                         self.subtotal += Lunch.menu_items['apps'].get(items)
#             if self.order.get('dessert') != None:
#                 for items in self.order.get('dessert').split(", "):
#                     if items in Lunch.menu_items['dessert']:
#                         self.subtotal += Lunch.menu_items['dessert'].get(items)
#         if self.menu == 'Dinner':
#             if self.order.get('mains') != None:
#                 for items in self.order.get('mains').split(", "):
#                     if items in Dinner.menu_items['mains']:
#                         self.subtotal += Dinner.menu_items['mains'].get(items)
#             if self.order.get('drinks') != None:
#                 for items in self.order.get('drinks').split(", "):
#                     if items in Dinner.menu_items['drinks']:
#                         self.subtotal += Dinner.menu_items['drinks'].get(items)
#             if self.order.get('apps') != None:
#                 for items in self.order.get('apps').split(", "):
#                     if items in Dinner.menu_items['apps']:
#                         self.subtotal += Dinner.menu_items['apps'].get(items)
#             if self.order.get('dessert') != None:
#                 for items in self.order.get('dessert').split(", "):
#                     if items in Dinner.menu_items['dessert']:
#                         self.subtotal += Dinner.menu_items['dessert'].get(items)


# V2
# if self.menu == 'Lunch':
#             for categories in ['mains','apps','drinks','dessert']:
#                 if self.order.get(categories) != None:
#                     for items in self.order.get(categories).split(", "):
#                         if items in Lunch.menu_items[categories]:
#                             self.subtotal += Lunch.menu_items[categories].get(items)
#         if self.menu == 'Dinner':
#             for categories2 in ['mains','apps','drinks','dessert']:
#                 if self.order.get(categories2) != None:
#                     for items in self.order.get(categories2).split(", "):
#                         if items in Dinner.menu_items[categories2]:
#                             self.subtotal += Dinner.menu_items[categories2].get(items)       

In [ ]:
# # START PROGRAM - 3/5 copy

#     def program(self):
#         print(self.customer_name)
#         print(self.menu['apps'])
#         bonus_item=''
# # Evaluates CC list. If first time, get a free introductory item. 
# #         def create_list(self):
#         credit_card_list = []
# #         credit_card_total = 0
#         for record in order_database:
#             credit_card_list.append(order_database[record]['Credit Card'])
# #         return credit_card_list
#         if credit_card_list.count(self.credit_card) == 1:
#             bonus_item = {'apps':'free introductory app'}
#             print('First time customer')
#             print(bonus_item)

# # If not the first time, calculate the sum of subtotals for that credit card in the past 100 days.
#         else:
# # def db_subtotal_agg(self,time=100):
#             credit_card_total = 0
#             for id_num, details in order_database.items():
#                 if details['Credit Card'] == self.credit_card and self.time > datetime.now() - timedelta(days=100):
#                     credit_card_total += details['Subtotal'] 
# #  If subtotal is greater than 350:
#             if credit_card_total > 50:
#                 temp_list = []
#                 for key, value in order_database.items():
#                     if value.get('Credit Card') == self.credit_card and self.time > datetime.now() - timedelta(days=200):
#                         tip = value.get('Order').get('apps').split(', ')
#                         temp_list.extend(tip)
#                 print('This a combined list of apps:') 
#                 print(temp_list)
#                 counter_dict = {}
#                 for item in temp_list:
#                     if counter_dict.get(item) == None:
#                         counter_dict[item]=1
#                     else:
#                         counter_dict[item] += 1
#                 self.counter = counter_dict
#                 print("Rolled Up:")
#                 print(self.counter)
# #                 print(self.order.get('apps'))
# # If app was ordered more than twice AND it is not on the current order, create a dictionary of compared items 
#                 comp_items = {}
#                 for key,value in self.counter.items():
#                     if value >2:
#                         if key in self.order.get('apps'):
#                             continue
#                         elif 'free' in key:
#                             continue
#                         elif key not in self.menu['apps'].keys():
#                             continue
#                         else: 
#                             comp_items[key] = value   
#                 print('Remove current order items, free items,and those less than 3')
#                 print(comp_items)
                
# # If there no frequent items NOT in the current order, send a dessert
#                 if comp_items == {}:
#                     bonus_item = {'dessert':'free sorbet'}
#                     print('Sending dessert bc no match')
#                     print(bonus_item)
                
# # If there is a list of frequent items. Get the most ordered item and return a dict. 
#                 else:
#                     for k,v in comp_items.items():
#                         if v == max(comp_items.values()):
#                             bonus_item = {'apps':k}
#                             bonus_item['apps'] = 'free '+str(k)
#                     print('Matched an item')
#                     print(bonus_item)
                    
# # Add to the database
# # def add_reward(self,added_reward):
#                     for key,value in bonus_item.items():
#                         order_database[self.orderid]['Order'][key] = value + ", " + self.order[key]
#                         print(order_database[self.orderid]['Order']['apps'])
# # return bonus_item
#             else:
#                 print('no rewards')
#         return 'Bonus item is', bonus_item